In [1]:
import math
from consts.geom import DEGREES_0, DEGREES_90, DEGREES_180, DEGREES_270, DEGREES_360
from consts.geom import DEGREES_EPSILON 

#pre_calc_tan_radians = {}
pre_calc_tan_radians = []

def calc_tan_radians(x):
    return math.tan(math.radians(x))

for x in range(0,int(90/DEGREES_EPSILON)):
    pre_calc_tan_radians.append(calc_tan_radians(x * DEGREES_EPSILON))

def get_tan_radians_for_angle(angle):
    index = int(angle / DEGREES_EPSILON)
    result = None
    try:
        result = pre_calc_tan_radians[index]    
    except IndexError as e:
        print("No value for angle:", angle, ", at index", index)
        raise e
        
    return result, index

for test_item in [1, 12, 27, 33, 42, 47, 54, 59, 66, 71, 79, 81, 89]:
    result, index = get_tan_radians_for_angle(test_item)
    assert(result == calc_tan_radians(test_item))

how much memory does this list take up in bytes?

In [2]:
from sys import getsizeof
getsizeof(pre_calc_tan_radians)

732816

360 degrees is tricky

In [3]:
def get_tan_radains(value):
    x_check = value    
    mod = 1
    
    if x_check >= DEGREES_360:
        x_check = x_check - DEGREES_360

    # TODO alternate between +/- depending on ray number
    if x_check in [DEGREES_0, DEGREES_90, DEGREES_180, DEGREES_270, DEGREES_360]:
        x_check = x_check - DEGREES_EPSILON
        
    x = x_check

    if x > DEGREES_0 and x < DEGREES_90:
        pass
    elif x > DEGREES_90 and x < DEGREES_180:
        x = (DEGREES_180 - DEGREES_90 - x) + DEGREES_90
        mod = -1
    elif x > DEGREES_180 and x < DEGREES_270:
        x = (x - DEGREES_180)
        mod = 1
    elif x > DEGREES_270 and x < DEGREES_360:
        x = (DEGREES_360 - DEGREES_270 - x) + DEGREES_270
        mod = -1
    
    result, index = get_tan_radians_for_angle(x)
    
    return (result * mod), x_check, index

In [4]:
def do_test(value):
    pre_calc_value, revised_x, index = get_tan_radains(value)
    calc_value = calc_tan_radians(revised_x)

    # check if equal to the decimal points of interest
    equal_to_points_of_interest = round(calc_value, 5) == round(pre_calc_value, 5)
    print(value, calc_value, pre_calc_value, equal_to_points_of_interest, index)
    assert(equal_to_points_of_interest)

In [5]:
test_values = [ 
    DEGREES_EPSILON, 0.1, 38, 55, 90, 100, 148, 167, 180, 190, 223, 249,
    261, 285, 305, 319, 339, 340, 285, 305, 319, 339, 340, 360
]
    
for test_value in test_values:
    do_test(test_value)

0.001 1.745329252171549e-05 1.745329252171549e-05 True 1
0.1 0.0017453310241888004 0.0017453310241888004 True 100
38 0.7812856265067174 0.7812856265067174 True 38000
55 1.4281480067421144 1.4281480067421144 True 55000
90 57295.779507212894 57295.779507212894 True 89999
100 -5.671281819617711 -5.671281819617707 True 80000
148 -0.6248693519093275 -0.6248693519093275 True 32000
167 -0.23086819112556334 -0.23086819112556312 True 13000
180 -1.7453292522014502e-05 -1.745329252171549e-05 True 1
190 0.1763269807084651 0.17632698070846498 True 10000
223 0.9325150861376613 0.9325150861376618 True 43000
249 2.605089064693798 2.6050890646938005 True 69000
261 6.313751514675036 6.313751514675041 True 81000
285 -3.7320508075688847 -3.7320508075688776 True 75000
305 -1.4281480067421146 -1.4281480067421144 True 55000
319 -0.8692867378162269 -0.8692867378162266 True 41000
339 -0.3838640350354164 -0.3838640350354158 True 21000
340 -0.3639702342662022 -0.36397023426620234 True 20000
285 -3.73205080756888

AssertionError: 

In [6]:
def calc_test():
    for n in range(1,26):
        for angle in range(0,361):
            result = calc_tan_radians(angle)

In [7]:
%timeit calc_test

10000000 loops, best of 3: 22.4 ns per loop


In [8]:
def pre_calc_test():
    for n in range(1,26):
        for angle in range(0,361):
            result = get_tan_radains(angle)

In [9]:
%timeit pre_calc_test()

100 loops, best of 3: 9.6 ms per loop


Looks like prebaked isn't quicker, perhaps if we get the index of the first ray and then slice the next n rays it could be comparitive?

In [10]:
def calc_test_rays(start_angle, number_of_rays):
    for i in (start_angle, start_angle + number_of_rays):
        calc_tan_radians(i)

In [11]:
def pre_calc_test_rays(start_angle, number_of_rays):
    pre_calc_value, revised_x, index = get_tan_radains(start_angle)
    return pre_calc_tan_radians[index:index+number_of_rays]


In [12]:
timeit calc_test_rays(225, 24)

The slowest run took 7.36 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 829 ns per loop


In [13]:
timeit pre_calc_test_rays(225, 24)

The slowest run took 5.07 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.38 µs per loop
